# Secop

## ¿Qué contratos del Estado se van a Cerrar?

Los contratos que requiere ejecutar el estado se publican en la portal de Colombia Compra Eficiente por medio de su plataforma SECOP II y los interesados en participar en la licitación, se registran y entran en el concurso de adjudicación.

Nuestro interés en este proyecto era predecir cuando un contrato adjudicado por el estado tenia la probabilidad más alta de cerrarse según las características de contrato como,  de que Departamento(Estado), el orden, la modalidad, el destino del gasto entre otros aspectos del relevantes de la contratación.

La diferencia entre un contrato cerrado y un contrato terminado es que un contrato cerrado es un contrato que ha sido ejecutado en su totalidad, liquidado y archivado, mientras que un contrato terminado es un contrato que ha sido ejecutado en su totalidad, pero que aún no ha sido liquidado o archivado.

**Un contrato cerrado es un contrato que ha finalizado y ya no está vigente y las partes no pueden realizar ninguna modificación al contrato y no pueden iniciar un proceso de reclamación de daños si alguna de las partes ha incumplido sus obligaciones contractuales**

## Datos

Los datos fuente fueron descargados del portal de Datos Abiertos del Gobierno de Colombia que están disponible a todo el público.

https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-II-Contratos-Electr-nicos/jbjy-vk9h

El archivo original pesa 3.62 Gb por lo tanto se tomo este archivo fue procesado inicialmente para descartar varias columnas de datos que no eran relevante para nuestra proyección y fue realizado en el notebook Filtrado_datos_secop generando un arhivo ideal (datos_filtradosv1.csv) para iniciar con el tratamiento de datos.

**Diccionario de datos**

https://www.datos.gov.co/api/views/jbjy-vk9h/files/839439f9-b3b9-4e53-a28d-ab82e752a1dc?download=true&filename=Diccionario%20de%20Datos%20Abiertos%202022%20Contratos%20Electronicos.pdf

### Importando archivo

Partimos del archivo "datos_filtradosv1.csv" para realizar el tratamiento de datos correspondiente para MLFlow

### Cargamos nuevo archivo filtrado

In [1]:
import mlflow

In [3]:
import pandas as pd

def load_secop(file):
    df = pd.read_csv(file)

    # Tomando columnas clave para el modelo
    df = df[['Departamento',
         'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
         'Estado Contrato',
         'Tipo de Contrato',
         'Modalidad de Contratacion',
         'Fecha de Firma', 'Fecha de Inicio del Contrato',
         'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
         'Fecha de Fin de Ejecucion',
         'Es Grupo', 'Es Pyme', 'Liquidación',
         'Obligación Ambiental',
         'Valor del Contrato', 'Valor Facturado',
         'Valor Pendiente de Pago', 'Valor Pagado',
         'Valor Pendiente de Ejecucion',
         'Saldo CDP',
         'EsPostConflicto', 'Destino Gasto',
         'Origen de los Recursos', 'Dias Adicionados',
         'Género Representante Legal',
         'Presupuesto General de la Nacion – PGN',
         'Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)',
         'Recursos Propios'
         ]]

    # Conversion de fechas 
    columnas_fecha = ['Fecha de Firma', 'Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato',
                  'Fecha de Inicio de Ejecucion', 'Fecha de Fin de Ejecucion']

    # Iterar sobre las columnas y convertirlas al tipo de dato 'date'
    for columna in columnas_fecha:
        df[columna] = pd.to_datetime(df[columna], format='%m/%d/%Y', errors='coerce')
        df[columna] = pd.to_datetime(df[columna], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    
    # Filtrado por estados
    estados_contrato = ["terminado", "Cerrado", "cedido", "Prorrogado", "Suspendido"]
    df_filtradoNoEjecucion = df[df["Estado Contrato"].isin(estados_contrato)]

    # FIltrado por año
    df_filtradoAnio = df_filtradoNoEjecucion[df_filtradoNoEjecucion['Fecha de Inicio del Contrato'].dt.year >= 2019]

    # Generar nuevas columnas
    df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'
    df_filtradoAnio = df_filtradoAnio.drop('Sector', axis=1)

    df_filtradoAnio["EsPrestacionServicios"] = df_filtradoAnio['Tipo de Contrato'] == 'Prestación de servicios'
    df_filtradoAnio = df_filtradoAnio.drop('Tipo de Contrato', axis=1)

    df_filtradoAnio["EsGrupo"] = df_filtradoAnio['Es Grupo'] == 'Si'
    df_filtradoAnio = df_filtradoAnio.drop('Es Grupo', axis=1)

    df_filtradoAnio["EsPyme"] = df_filtradoAnio['Es Pyme'] == 'Si'
    df_filtradoAnio = df_filtradoAnio.drop('Es Pyme', axis=1)

    df_filtradoAnio["EstaLiquidado"] = df_filtradoAnio['Liquidación'] == 'Si'
    df_filtradoAnio = df_filtradoAnio.drop('Liquidación', axis=1)

    df_filtradoAnio["EsObligacionAmbiental"] = df_filtradoAnio['Obligación Ambiental'] == 'Si'
    df_filtradoAnio = df_filtradoAnio.drop('Obligación Ambiental', axis=1)

    df_filtradoAnio["Es PostConflicto"] = df_filtradoAnio['EsPostConflicto'] == 'Si'
    df_filtradoAnio = df_filtradoAnio.drop('EsPostConflicto', axis=1)

    df_filtradoAnio["EsRecursosPropios"] = df_filtradoAnio['Origen de los Recursos'] == 'Recursos Propios'
    df_filtradoAnio = df_filtradoAnio.drop('Origen de los Recursos', axis=1)

    # Borrando duplicados
    df_filtradoAnio = df_filtradoAnio.drop_duplicates()

    # Aplicar one-hot encoding
    df_one_hot = pd.get_dummies(df_filtradoAnio['Estado Contrato'])

    # Combinar el DataFrame original con las nuevas columnas
    df_final = pd.concat([df_filtradoAnio, df_one_hot], axis=1)

    # Eliminar la columna original "Estado Contrato" si deseas
    df_final = df_final.drop('Estado Contrato', axis=1)

    return df_final

### Separa columnas y divide el dataSet

In [4]:
from sklearn.model_selection import train_test_split

def split_dataset(df_final, train_proportion, test_proportion):
    estado_contrato = df_final["Cerrado"].copy()
    contratos_data = df_final.drop(["terminado","cedido","Suspendido","Cerrado"],axis=1)
    # Ordenando DataSet
    contratos_data = contratos_data[["Departamento","Orden","Entidad Centralizada","Modalidad de Contratacion","Destino Gasto","Género Representante Legal","EsServicioPublico","EsRecursosPropios","EsGrupo","EsPrestacionServicios","EsPyme","EstaLiquidado","EsObligacionAmbiental","Es PostConflicto","Dias Adicionados","Valor del Contrato","Valor Facturado","Valor Pendiente de Pago","Saldo CDP"]]

    # Calculate test and validation set size:
    original_count = len(df_final)
    training_size = int(original_count * train_proportion)
    test_size = int((1 - train_proportion) * test_proportion * training_size)

    train_x, rest_x, train_y, rest_y = train_test_split(contratos_data, estado_contrato, train_size=training_size)
    test_x, validate_x, test_y, validate_y = train_test_split(rest_x, rest_y, train_size=test_size)

    mlflow.log_params({
        'dataset_size': original_count,
        'training_set_size': len(train_x),
        'validate_set_size': len(validate_x),
        'test_set_size': len(test_x)
    })

    return (train_x, train_y), (validate_x, validate_y), (test_x, test_y)

### Codificando Variables - Pipeline

In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.ensemble import RandomForestClassifier

def build_pipeline(n_estimators, max_depth):
    # OneHotEncoder
    internal_one_hot_encoding = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    columns_to_encode = [
        "Orden",
        "Entidad Centralizada",
        "Modalidad de Contratacion",
        "Destino Gasto",
        "Género Representante Legal",
        "Departamento"
        ]

    #Registrando OneHotEncoder en el log_param de mlflow
    mlflow.log_param('one_hot_encoded_columns', columns_to_encode)
    encoder_params = internal_one_hot_encoding.get_params()
    mlflow.log_params({
        f"encoder__{key}": value for key, value in encoder_params.items()
    })

    one_hot_encoding = ColumnTransformer([
        (
            'one_hot_encode',
            internal_one_hot_encoding,
            columns_to_encode
        )
    ])

    #Binarizer
    internal_binarizer = Binarizer()
    internal_encoder_binarizer = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    columns_to_binarize = [
        "EsServicioPublico",
        "EsRecursosPropios",
        "EsGrupo",
        "EsPrestacionServicios",
        "EsPyme",
        "EstaLiquidado",
        "EsObligacionAmbiental",
        "Es PostConflicto"
        ]
    
    #Registrando Binarizer en el log_param de mlflow
    mlflow.log_param('binarized_columns', columns_to_binarize)
    binarizer_params = internal_encoder_binarizer.get_params()
    mlflow.log_params({
        f"encoder__{key}": value for key, value in binarizer_params.items()
    })

    binarizer = ColumnTransformer([
        (
            'binarizer',
            internal_binarizer,
            columns_to_binarize
        )
    ])

    one_hot_binarized = Pipeline([
        ("binarizer", binarizer),
        ("one_hot_encoder", internal_encoder_binarizer),
    ])

    #Scaler de valores
    internal_scaler = RobustScaler()
    columns_to_scale = ["Valor del Contrato","Valor Facturado","Valor Pendiente de Pago","Saldo CDP"]

    #Registrando scaler en el log_param de mlflow
    mlflow.log_param('scaled_columns', columns_to_scale)
    scaler_params = internal_scaler.get_params()
    mlflow.log_params({
        f"encoder__{key}": value for key, value in scaler_params.items()
    })

    scaler = ColumnTransformer([
        ("scaler", internal_scaler, columns_to_scale)
    ])

    #Passthrough
    pass_columns = ["Dias Adicionados"]
    #Registrando passthrough en el log_param de mlflow
    mlflow.log_param('passthrough_columns', pass_columns)

    passthrough = ColumnTransformer([
        (
            "passthrough",
            "passthrough",
            pass_columns
        )
    ])

    feature_engineering_pipeline = pipe = Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("categorical", one_hot_encoding),
                        ("categorical_binarized", one_hot_binarized),
                        ("scaled", scaler),
                        ("pass", passthrough)
                    ]
                ),
            )
        ]
    )

    # ML model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    model_params = model.get_params()
    mlflow.log_params({
        f"model__{key}": value for key, value in model_params.items()
    })

    final_pipeline = Pipeline([
        ("feature_engineering", feature_engineering_pipeline),
        ("model", model)
    ])

    return final_pipeline

### Model Training and Validation

In [6]:
from sklearn.metrics import accuracy_score, recall_score

def model_training_validation(final_pipeline, train_x, train_y, validate_x, validate_y):
    final_pipeline.fit(train_x, train_y)

    train_pred_y = final_pipeline.predict(train_x)
    validate_pred_y = final_pipeline.predict(validate_x)

    train_accuracy = accuracy_score(train_pred_y, train_y)
    train_recall = recall_score(train_pred_y, train_y)
    
    validate_accuracy = accuracy_score(validate_pred_y, validate_y)
    validate_recall = recall_score(validate_pred_y, validate_y)

    print('Train accuracy', train_accuracy)
    print('Train recall', train_recall)
    
    print('Validate accuracy', validate_accuracy)
    print('Validate recall', validate_recall)

    metrics = {
        'train_accuracy': train_accuracy,
        'train_recall': train_recall,
        'validate_accuracy': validate_accuracy,
        'validate_recall': validate_recall,
    }

    mlflow.log_metrics(metrics)

    return final_pipeline

### Run Full Training

In [21]:
from joblib import dump
from mlflow.models.signature import infer_signature

def full_training_run():

    mlflow.set_experiment("Contratos_SECOP")

    with mlflow.start_run() as run:

        raw_dataset = load_secop(r"../data/processed/datos_filtradosv1.csv")    
        training_data, validate_data, test_data = split_dataset(raw_dataset, train_proportion=0.6, test_proportion=0.5)        
        training_pipeline = build_pipeline(n_estimators=100, max_depth=100)        
        training_pipeline = model_training_validation(
            training_pipeline,
            train_x=training_data[0],
            train_y=training_data[1],
            validate_x=validate_data[0],
            validate_y=validate_data[1]
        )

        dump(training_pipeline, "Secop_pipeline.joblib", compress=5)

        mlflow.log_artifact('Secop_pipeline.joblib')

        signature = infer_signature(training_data[0], training_data[1])
        mlflow.sklearn.log_model(
            sk_model=training_pipeline,
            artifact_path="secop-model",
            signature=signature,
            registered_model_name="SecopModel",
        )

        !mv "Secop_pipeline.joblib" "../models/"

    return training_pipeline

In [22]:
full_training_run()

/var/folders/j_/_xz8f5pj26n0_6fzf8ktr7xw0000gn/T/ipykernel_3534/2136726870.py:4: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/j_/_xz8f5pj26n0_6fzf8ktr7xw0000gn/T/ipykernel_3534/2136726870.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'


Train accuracy 0.9840646272949
Train recall 0.9777418956086406
Validate accuracy 0.848376507296779
Validate recall 0.8169976253570568


/Users/rdiaz/Library/CloudStorage/OneDrive-Personal/10_Codigofacilito/02_Bootcamp_Ciencia Datos/10_Project/secop_project/venv/lib/python3.11/site-packages/mlflow/models/signature.py:152: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)
/Users/rdiaz/Library/CloudStorage/OneDrive-Personal/10_Codigofacilito/02_Bootcamp_Ciencia Datos/10_Project/

Pipeline(steps=[('feature_engineering',
                 Pipeline(steps=[('features',
                                  FeatureUnion(transformer_list=[('categorical',
                                                                  ColumnTransformer(transformers=[('one_hot_encode',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse_output=False),
                                                                                                   ['Orden',
                                                                                                    'Entidad '
                                                                                                    'Centralizada',
                                                                                                    'Modalidad '
                                                                                                    'de '
                                                                                                    'Contratacion',
                                                                                                    'Destino '
                                                                                                    'Gasto',
                                                                                                    'Género '
                                                                                                    'Representante '
                                                                                                    'Legal',
                                                                                                    'Departamento'])]...
                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                 sparse_output=False))])),
                                                                 ('scaled',
                                                                  ColumnTransformer(transformers=[('scaler',
                                                                                                   RobustScaler(),
                                                                                                   ['Valor '
                                                                                                    'del '
                                                                                                    'Contrato',
                                                                                                    'Valor '
                                                                                                    'Facturado',
                                                                                                    'Valor '
                                                                                                    'Pendiente '
                                                                                                    'de '
                                                                                                    'Pago',
                                                                                                    'Saldo '
                                                                                                    'CDP'])])),
                                                                 ('pass',
                                                                  ColumnTransformer(transformers=[('passthrough',
                                                                                                   'passthrough',
                                         